In [1]:
%load_ext sparkmagic.magics

In [2]:
import os
from IPython import get_ipython
username = os.environ['RENKU_USERNAME']
server = "http://iccluster029.iccluster.epfl.ch:8998"

# set the application name as "<your_gaspar_id>-homework3"
get_ipython().run_cell_magic(
    'spark',
    line='config', 
    cell="""{{ "name": "{0}-homework3", "executorMemory": "4G", "executorCores": 4, "numExecutors": 10, "driverMemory": "4G"}}""".format(username)
)

In [3]:
get_ipython().run_line_magic(
    "spark", "add -s {0}-homework3 -l python -u {1} -k".format(username, server)
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5348,application_1649870447656_1949,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [4]:
%%spark
print('We are using Spark %s' % spark.version)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We are using Spark 2.3.2.3.1.4.0-315

In [5]:
%%spark?

Docstring:
::

  %spark [-c CONTEXT] [-s SESSION] [-o OUTPUT] [-q [QUIET]]
             [-m SAMPLEMETHOD] [-n MAXROWS] [-r SAMPLEFRACTION] [-u URL]
             [-a USER] [-p PASSWORD] [-t AUTH] [-l LANGUAGE] [-k [SKIP]]
             [-i ID] [-e COERCE]
             [command ...]

Magic to execute spark remotely.

This magic allows you to create a Livy Scala or Python session against a Livy endpoint. Every session can
be used to execute either Spark code or SparkSQL code by executing against the SQL context in the session.
When the SQL context is used, the result will be a Pandas dataframe of a sample of the results.

If invoked with no subcommand, the cell will be executed against the specified session.

Subcommands
-----------
info
    Display the available Livy sessions and other configurations for sessions.
add
    Add a Livy session given a session name (-s), language (-l), and endpoint credentials.
    The -k argument, if present, will skip adding this session if it already exist

### Import data

In [6]:
%%spark
all_stops = spark.read.orc('/data/sbb/orc/allstops')
all_stops.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- stop_id: string (nullable = true)
 |-- stop_name: string (nullable = true)
 |-- stop_lat: double (nullable = true)
 |-- stop_lon: double (nullable = true)
 |-- location_type: string (nullable = true)
 |-- parent_station: string (nullable = true)

In [29]:
%%spark
actual_data = spark.read.orc('/data/sbb/orc/istdaten')
actual_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- betriebstag: string (nullable = true)
 |-- fahrt_bezeichner: string (nullable = true)
 |-- betreiber_id: string (nullable = true)
 |-- betreiber_abk: string (nullable = true)
 |-- betreiber_name: string (nullable = true)
 |-- produkt_id: string (nullable = true)
 |-- linien_id: string (nullable = true)
 |-- linien_text: string (nullable = true)
 |-- umlauf_id: string (nullable = true)
 |-- verkehrsmittel_text: string (nullable = true)
 |-- zusatzfahrt_tf: string (nullable = true)
 |-- faellt_aus_tf: string (nullable = true)
 |-- bpuic: string (nullable = true)
 |-- haltestellen_name: string (nullable = true)
 |-- ankunftszeit: string (nullable = true)
 |-- an_prognose: string (nullable = true)
 |-- an_prognose_status: string (nullable = true)
 |-- abfahrtszeit: string (nullable = true)
 |-- ab_prognose: string (nullable = true)
 |-- ab_prognose_status: string (nullable = true)
 |-- durchfahrt_tf: string (nullable = true)

### Sub-sample

Sub-sample data for testing.

In [30]:
%%spark

test_mode = True
actual_data = actual_data.sample(0.001)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Rename columns

Columns are renamed in English as follows:
- `betriebstag` to DATE
- `fahrt_bezeichner` to TRIP_ID
- `produkt_id` to PRODUCT_ID
- `haltestellen_name` to STOP_NAME
- `ankunftszeit` to ARRIVAL_TIME
- `an_prognose` to ACTUAL_ARRIVAL_TIME
- `abfahrtszeit` to DEPARTURE_TIME
- `ab_prognose` to ACTUAL_DEPARTURE_TIME

In [31]:
%%spark

#rename columns in english

actual_data = actual_data.select(actual_data.betriebstag.alias('DATE'), actual_data.fahrt_bezeichner.alias('TRIP_ID'), actual_data.produkt_id.alias('PRODUCT_ID'), actual_data.haltestellen_name.alias('STOP_NAME'), actual_data.ankunftszeit.alias('ARRIVAL_TIME'), actual_data.an_prognose.alias('ACTUAL_ARRIVAL_TIME'), actual_data.abfahrtszeit.alias('DEPARTURE_TIME'), actual_data.ab_prognose.alias('ACTUAL_DEPARTURE_TIME'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Prepare table

In [129]:
%%spark
import pyspark.sql.functions as F
# F.date_format(F.to_timestamp(df_used.timestamp_s), 'yyyy-MM-dd HH:mm:ss').alias('date'))
df = actual_data.withColumn('DATE', F.date_format(F.to_timestamp(actual_data.DATE, 'dd.MM.yyyy'), 'yyyy-MM-dd'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
%%spark
import pyspark.sql.functions as F

df = df.withColumn('DAY_OF_WEEK', ((F.dayofweek(df['DATE'])+5)%7)+1)
df = df.withColumn('WEEK_OF_YEAR', F.weekofyear(df['DATE']))
df = df.withColumn('MONTH_OF_YEAR', F.month(df['DATE']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
%%spark
import pyspark.sql.functions as F

df = df.withColumn('ARRIVAL_TIME', F.unix_timestamp(df.ARRIVAL_TIME, 'dd.MM.yyyy HH:mm'))
df = df.withColumn('ACTUAL_ARRIVAL_TIME', F.unix_timestamp(df.ACTUAL_ARRIVAL_TIME, 'dd.MM.yyyy HH:mm'))
df = df.withColumn('DEPARTURE_TIME', F.unix_timestamp(df.DEPARTURE_TIME, 'dd.MM.yyyy HH:mm'))
df = df.withColumn('ACTUAL_DEPARTURE_TIME', F.unix_timestamp(df.ACTUAL_DEPARTURE_TIME, 'dd.MM.yyyy HH:mm'))


df = df.withColumn('ARRIVAL_DELAY', F.when((df['ACTUAL_ARRIVAL_TIME']-df['ARRIVAL_TIME'] )< 0, 0).otherwise(df['ACTUAL_ARRIVAL_TIME']-df['ARRIVAL_TIME']))
df = df.withColumn('DEPARTURE_DELAY', F.when((df['ACTUAL_DEPARTURE_TIME']-df['DEPARTURE_TIME'] )< 0, 0).otherwise(df['ACTUAL_DEPARTURE_TIME']-df['DEPARTURE_TIME']))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
%%spark
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DATE: string (nullable = true)
 |-- TRIP_ID: string (nullable = true)
 |-- PRODUCT_ID: string (nullable = true)
 |-- STOP_NAME: string (nullable = true)
 |-- ARRIVAL_TIME: long (nullable = true)
 |-- ACTUAL_ARRIVAL_TIME: long (nullable = true)
 |-- DEPARTURE_TIME: long (nullable = true)
 |-- ACTUAL_DEPARTURE_TIME: long (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- WEEK_OF_YEAR: integer (nullable = true)
 |-- MONTH_OF_YEAR: integer (nullable = true)
 |-- ARRIVAL_DELAY: long (nullable = true)
 |-- DEPARTURE_DELAY: long (nullable = true)

### Filter data based on simplifying assumptions

We only consider journeys at reasonable hours of the day, and on a typical business day.
- `day`: from Monday to Friday
- `hours`: first departure at 6:00, last arrival at 21:00

In [151]:
%%spark

#business day
df = df.filter(df['DAY_OF_WEEK'] <6)
#reasonable hours of the day
df = df.filter(F.hour(F.from_unixtime(df['DEPARTURE_TIME']))>=6)
df = df.filter(F.hour(F.from_unixtime(df['ARRIVAL_TIME']))<=21)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Compute condifence given threshold

In [126]:
%%spark

threshold = 60 #seconds

df = df.withColumn('ARRIVAL_THRESHOLD', F.when(df['ARRIVAL_DELAY']<=threshold, 1).otherwise(0))
df = df.withColumn('DEPARTURE_THRESHOLD', F.when(df['DEPARTURE_DELAY']<=threshold, 1).otherwise(0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [127]:
%%spark

df2 = df.groupBy(df['DAY_OF_WEEK'], df['WEEK_OF_YEAR'], df['MONTH_OF_YEAR'], df['STOP_NAME'], df['TRIP_ID']).agg(F.mean('ARRIVAL_THRESHOLD').alias('ARRIVAL_CONFIDENCE'), F.mean('DEPARTURE_THRESHOLD').alias('DEPARTURE_CONFIDENCE'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [156]:
%%spark
df.select(F.from_unixtime(df['ARRIVAL_TIME'])).sample(False, 0.1, seed=31).first()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(from_unixtime(ARRIVAL_TIME, yyyy-MM-dd HH:mm:ss)=u'2019-09-10 06:58:00')